In [73]:
import torch
from torchvision.transforms import transforms
import torch.nn as nn
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader
import pathlib 
from pathlib import Path

In [74]:
class CatDog(Dataset):

    def __init__(self, dataset_path: pathlib.Path):
        self.dataset_path = Path(dataset_path)
        self.image_paths = self.dataset_path.glob("*.jpg") 

        # self.data = {"cats" : [], "dogs" : []}

        # for ip in self.image_paths:
        #     if "cat" == ip.name[:3]:
        #         self.data["cats"].append(ip.read_bytes())
        #     elif "dog" == ip.name[:3]:
        #         self.data["dogs"].append(ip.read_bytes())

        # Legacy support
        self.data_list = list(self.image_paths)
        # self.data_class = ["cat", "dog"]

        self.transform_func = transforms.Compose([
             transforms.ToPILImage(),
             transforms.Grayscale(),
             transforms.Resize((124, 124)),
             transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, index):
        img_path = self.data_list[index]
        img_label = 0 if self.data_list[index].name[:3] == "cat" else 1
        img_tensor = read_image(str(img_path))
        return self.transform_func(img_tensor), img_label
    

### Дебажные клетки на случай ненахода датасета нейронкой

In [75]:
dataset_path = Path("./train/")
image_paths = dataset_path.glob("*.jpg") 
data_list = list(image_paths)
len(data_list)

25000

In [76]:
for p in data_list[:10]:
    print(p.name)

cat.11338.jpg
dog.9276.jpg
cat.8034.jpg
dog.5303.jpg
cat.11569.jpg
cat.12272.jpg
cat.4679.jpg
dog.6976.jpg
dog.9670.jpg
cat.3341.jpg


In [109]:
class CVModel(nn.Module):

    def __init__(self):
        super(CVModel, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, (3, 3))
        self.conv_2 = nn.Conv2d(32, 64, (3, 3))
        self.pool_1 = nn.MaxPool2d((2, 2))
        self.conv_3 = nn.Conv2d(64, 128, (3, 3))
        self.conv_4 = nn.Conv2d(128, 128, (3, 3))
        self.pool_2 = nn. MaxPool2d((2, 2))
        self.glob_pool = nn.MaxPool2d((28, 28))
        self.linear = nn.Linear(128, 2) #2 классa
        self.softmax = nn.Softmax()

    def forward(self, X):
        res = self.conv_1(X)
        res = self.conv_2(res)
        res = self.pool_1(res)
        res = self.conv_3(res)
        res = self.conv_4(res)
        res = self.pool_2(res)
        res = self.glob_pool(res) #(1, 1, 128)
        res = self.linear(res.view(-1, 128)) #Сокращение размерности до (128)
        return self.softmax(res)

In [171]:
path = pathlib.Path(r'./train/')
loss = nn.CrossEntropyLoss() #работает только со скалярами (либо просто энтропию)
dataset = CatDog(path)
model = CVModel()
trainloader = DataLoader(dataset, batch_size=32, shuffle=True)
optimizer = torch.optim.Adam(params=model.parameters())

In [174]:
def train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset):
    correct = 0
    epoch_loss = 0.0 
    for i, data in enumerate(trainloader, 0):
        x, y = data
        optimizer.zero_grad()
        outputs = model(x)
        error = loss(outputs, y)
        error.backward()
        epoch_loss += outputs.shape[0] * error.item()
        optimizer.step()
        outputs = (outputs > 0.5).float()
        # print(f"outputs: {outputs.shape}\n\ny: {y.shape}")
        correct += (outputs == y).float().sum()

    print(f'TRAIN [{epoch + 1}] loss: {epoch_loss/ dataset.__len__():.3f}')

In [175]:
for epoch in range(1):
    train_one_epoch(epoch, trainloader, optimizer, model, loss, dataset)

RuntimeError: The size of tensor a (2) must match the size of tensor b (32) at non-singleton dimension 1